# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [17]:
import pandas as pd
import boto3
import sagemaker
from sagemaker import image_uris
from sagemaker import LinearLearner
import os

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [22]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism'
output_path = 's3://{}/{}/'.format(bucket, prefix)
print('Model artifacts are stored in ', output_path)

train_key = os.path.join(data_dir, 'train.csv')
test_key = os.path.join(data_dir, 'test.csv')
train_prefix = '{}/{}'.format(prefix, 'train')
test_prefix = '{}/{}'.format(prefix, 'test')

# upload all data to S3
s3_train_data = sagemaker_session.upload_data(path=train_key, bucket=bucket, key_prefix=train_prefix)
s3_test_data = sagemaker_session.upload_data(path=test_key, bucket=bucket, key_prefix=test_prefix)

# check locations for data in s3
print('Training data is stored in: ', s3_train_data)
print('Test data is stored in: ', s3_test_data)

Model artifacts are stored in  s3://sagemaker-us-west-2-904606187431/plagiarism/
Training data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/train/train.csv
Test data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/test/test.csv
Training Y data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/train/train_y.csv
Test Y data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/test/test_y.csv
Training X data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/train/train_x.csv
Test X data is stored in:  s3://sagemaker-us-west-2-904606187431/plagiarism/test/test_x.csv


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
#     print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [ ]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

# Define a PyTorch Estimator

In [1]:
from source_pytorch.model import BinaryClassifier
test_model = BinaryClassifier(8, 25, 1)
print(test_model)

BinaryClassifier(
  (fc1): Linear(in_features=8, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=1, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
  (sigmoid): Sigmoid()
)


In [3]:
!pygmentize source_pytorch/model.py

# torch imports
import torch
import torch.nn.functional as F
import torch.nn as nn


## TODO: Complete this classifier
class BinaryClassifier(nn.Module):
    """ Define a neural network that performs binary classification.
        The network should accept your number of features as input, and produce 
        a single sigmoid value, that can be rounded to a label: 0 or 1, as output.
    
        Notes on training:
        To train a binary classifier in PyTorch, use BCELoss.
        BCELoss is binary cross entropy loss, documentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss
    """

    ## Define the init function, the input params are required (for loading code in train.py to work)
    def __init__(self, input_features, hidden_dim, output_dim):
        """ Initialize the model by setting up linear layers.
            Use the input parameters to help define the layers of your model.
            
            Parameters:
                input_features: the number of inp

# Train the PyTorch Estimator

In [ ]:
from sagemaker.pytorch import PyTorch

# instantiate a PyTorch estimator
# use same output_path as other estimator
pytorch_estimator = PyTorch(entry_point='train.py',
                            source_dir='source_pytorch',
                            role=role,
                            framework_version='2.50', # need to confirm this before training
                            output_path=output_path,
                            instance_count=1,
                            instance_type='ml.c4.xlarge',
                            sagemaker_session=sagemaker_session,
                            hyperparameters={
                                'input_features': 8,
                                'hidden_dim': 25,
                                'output_dim': 1,
                                'epochs': 25
                            })

# Deploy the Pytorch Model

# Evaluate the Pytorch Model

# Delete Pytorch Model Endpoint

## Sagemaker - LinearLearner

In [27]:
# convert data to RecordSet
train_data = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)
train_data.head()

,0,1,2,3,4,5,6,7,8
0,0,0.009346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.191781
1,1,0.613636,0.515982,0.449541,0.382488,0.319444,0.265116,0.219626,0.846491
2,1,0.156757,0.108696,0.081967,0.060440,0.044199,0.027778,0.011173,0.316062
3,0,0.031746,0.005319,0.000000,0.000000,0.000000,0.000000,0.000000,0.242574
4,0,0.007722,0.003876,0.000000,0.000000,0.000000,0.000000,0.000000,0.161172


In [28]:
train_x_np = train_data.iloc[:,1:].to_numpy().astype('float32')
train_y_np = train_data.iloc[:,0].to_numpy().astype('float32')

In [29]:
# your import and estimator code, here
region = sagemaker_session.boto_region_name
print(region)

linear_learner = LinearLearner(role=role,
                               instance_count=1,
                               instance_type='ml.m4.xlarge',
                               sagemaker_session=sagemaker_session,
                               output_path=output_path,
                               predictor_type='binary_classifier',
                               epochs=25)

us-west-2


In [31]:
# create RecordSet
formatted_train_data = linear_learner.record_set(train_x_np, labels=train_y_np)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [32]:
%%time

# Train your estimator on S3 training data
linear_learner.fit(formatted_train_data)


Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-07-31 17:48:38 Starting - Starting the training job...
2021-07-31 17:48:40 Starting - Launching requested ML instancesProfilerReport-1627753718: InProgress
......
2021-07-31 17:50:07 Starting - Preparing the instances for training.........
2021-07-31 17:51:33 Downloading - Downloading input data...
2021-07-31 17:51:53 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/31/2021 17:52:22 INFO 140413398746944] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer':

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

# Deploy LinearLearner

In [33]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel

# deploy your model to create a predictor
linear_predictor = linear_learner.deploy(initial_instance_count=1,
                                         instance_type='ml.t2.medium')


Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------!CPU times: user 307 ms, sys: 25.7 ms, total: 332 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [36]:
test_y_np = test_y.to_numpy().astype('float32')
test_x_np = test_x.to_numpy().astype('float32')

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [44]:
import numpy as np

test_y_preds = linear_predictor.predict(test_x_np)
test_y_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in test_y_preds])

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [45]:
from sklearn.metrics import accuracy_score

# Second: calculate the test accuracy
accuracy = accuracy_score(test_y_np, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [46]:
def accuracy_metrics(predictor, test_data_path, verbose=True):
    """
        Evaluate a model on a test set given the prediction endpoint.  
        Return binary classification metrics.
        
        Params
            predictor: A prediction endpoint
            test_data: Test features and labels
            verbose: If True, prints a table of all performance metrics
        Return
            A dictionary of performance metrics.
    """
    # convert the test data to correct format
    test_data = pd.read_csv(test_data_path, header=None, names=None)
    test_y = test_data.iloc[:,0]
    test_x = test_data.iloc[:,1:]
    test_y_np = test_y.to_numpy().astype('float32')
    test_x_np = test_x.to_numpy().astype('float32')
    
    # get predictions from predictor
    test_y_preds = predictor.predict(test_x_np)
    test_y_preds = np.array([x.label['predicted_label'].float32_tensor.values[0] for x in test_y_preds])
    
    # calculate true positives, false positives, true negatives, and false negatives
    tp = np.logical_and(test_y_np, test_y_preds).sum()
    fp = np.logical_and(1-test_y_np, test_y_preds).sum()
    tn = np.logical_and(1-test_y_np, 1-test_y_preds).sum()
    fn = np.logical_and(test_y_np, 1-test_y_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print results 
    if verbose:
        print(pd.crosstab(test_y_np, test_y_preds, rownames=['Actual (row)'], colnames=['Prediction (col)']))
        print('\n{:<11} {:.3f}'.format('Recall:', recall))
        print('\n{:<11} {:.3f}'.format('Precision:', precision))
        print('\n{:<11} {:.3f}'.format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp,
            'FP': fp,
            'FN': fn,
            'TN': tn,
            'Precision': precision,
            'Recall': recall,
            'Accuracy': accuracy}

In [47]:
test_data_path = os.path.join(data_dir, 'test.csv')
accuracy_metrics(linear_predictor, test_data_path)

Prediction (col)  0.0  1.0
Actual (row)              
0.0                 9    1
1.0                 0   15

Recall:     1.000

Precision:  0.938

Accuracy:   0.960



{'TP': 15,
 'FP': 1,
 'FN': 0,
 'TN': 9,
 'Precision': 0.9375,
 'Recall': 1.0,
 'Accuracy': 0.96}

### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
The model produced _one_ false positive and _zero_ false negatives. The value of false positives produced is most likely due to the fact that the model either needs more training runs or more features. However, the fact that there are _zero_ false negatives is an indication that the model had not been over fitted. The accuracy score is fairly good indicating that the model is performing well. 

### Question 2: How did you decide on the type of model to use? 

** Answer**:
I decided to use the LinearLearner from Sagemaker because the data does not have too many features. This low dimensional feature space is suited well for a binary classifer such as the LinearLearner. If more features were present then perhaps a more robust model such as a Random Forest classifier would be better suited. I also suspect that since the data set is not that large that another model was not as necessary.


----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [48]:
# uncomment and fill in the line below!
linear_learner.delete_endpoint()

The function delete_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [ ]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!